In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [ ]:
df.columns

# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values. feature importance was used to select the more effective ones.
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 
        'koi_impact',
        'koi_duration',
        'koi_depth', 
        'koi_prad', 
       'koi_teq', 'koi_insol', 
       'koi_model_snr'
         
         
       ]]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
# define the output as target
target = df["koi_disposition" ]


In [7]:
selected_features_dummies = pd.get_dummies(selected_features)


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(selected_features_dummies, target, random_state=42)


In [ ]:
 y_train.head()

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [9]:
# Scale your data

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
X_scaler = MinMaxScaler().fit(X_train)


X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



In [17]:
# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [18]:
# Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, y_train_categorical)
rf.score(X_test_scaled, y_test_categorical)

0.8941647597254004

In [20]:
predicted = rf.predict(X_test_scaled)



In [14]:
# check the importance of the features and remove the least important ones
sorted(zip(rf.feature_importances_, selected_features_dummies), reverse=True)

[(0.1543347151778402, 'koi_model_snr'),
 (0.14287404281128405, 'koi_fpflag_nt'),
 (0.13688408316263126, 'koi_fpflag_co'),
 (0.11039153511732179, 'koi_fpflag_ss'),
 (0.09320939233447448, 'koi_prad'),
 (0.06734747267696667, 'koi_depth'),
 (0.06311441896062414, 'koi_period'),
 (0.05518637566380634, 'koi_impact'),
 (0.046640886739842104, 'koi_teq'),
 (0.04613807261106618, 'koi_duration'),
 (0.04505206798628156, 'koi_fpflag_ec'),
 (0.03882693675786115, 'koi_insol')]

# Train the Model



In [21]:
print(f"Training Data Score: {rf.score(X_train_scaled, y_train_categorical)}")
print(f"Testing Data Score: {rf.score(X_test_scaled, y_test_categorical)}")

Training Data Score: 1.0
Testing Data Score: 0.8941647597254004


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [22]:
# Create the GridSearchCV model

from sklearn.model_selection import GridSearchCV
param_grid = {'min_samples_split': [2, 5, 10],
              'bootstrap': [ True],
             'n_estimators': [100]}
grid = GridSearchCV(rf, param_grid, verbose=3)

In [23]:
# Train the model with GridSearch.Several different parameters were tested
grid.fit(X_train_scaled, y_train_categorical)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END bootstrap=True, min_samples_split=2, n_estimators=100; total time=   1.1s
[CV 2/5] END bootstrap=True, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV 3/5] END bootstrap=True, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV 4/5] END bootstrap=True, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV 5/5] END bootstrap=True, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV 1/5] END bootstrap=True, min_samples_split=5, n_estimators=100; total time=   1.0s
[CV 2/5] END bootstrap=True, min_samples_split=5, n_estimators=100; total time=   1.0s
[CV 3/5] END bootstrap=True, min_samples_split=5, n_estimators=100; total time=   1.0s
[CV 4/5] END bootstrap=True, min_samples_split=5, n_estimators=100; total time=   1.0s
[CV 5/5] END bootstrap=True, min_samples_split=5, n_estimators=100; total time=   1.0s
[CV 1/5] END bootstrap=True, min_samples_split=10, n_estimators=100; t

GridSearchCV(estimator=RandomForestClassifier(n_estimators=200),
             param_grid={'bootstrap': [True], 'min_samples_split': [2, 5, 10],
                         'n_estimators': [100]},
             verbose=3)

In [ ]:
# rf.get_params().keys()

In [24]:
print(grid.best_params_)
print(grid.best_score_)

{'bootstrap': True, 'min_samples_split': 5, 'n_estimators': 100}
0.888419905544357


In [25]:
rf = grid

In [27]:
rf.predict (X_test_scaled)

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

# Save the Model

In [28]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'fatmabutun_rf.sav'
joblib.dump(rf, filename)

['fatmabutun_rf.sav']